Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [74]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from tqdm import tqdm

## The *Nim* and *Nimply* classes

In [3]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [99]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        if self._rows[row] < num_objects :
            print(self, row, num_objects)
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [5]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [6]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [7]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [113]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)

# Use a cache to improve computation time of this function
analize_cache = dict()
def analize(raw: Nim) -> dict:
    global analize_cache
    cached_value = analize_cache.get(raw)
    if cached_value is not None:
        key = list(analize_cache.keys())[list(analize_cache.values()).index(cached_value)]
        print(raw, key)
        return cached_value
    
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    analize_cache[raw] = cooked
    print(raw, "added in cache -->", cooked)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [47]:
logging.getLogger().setLevel(logging.INFO)

strategy = (pure_random, optimal)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=2)
INFO:root:status: <1 1 5 7 9>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=2)
INFO:root:status: <1 1 3 7 9>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <1 0 3 7 9>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=1)
INFO:root:status: <1 0 3 7 8>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=7)
INFO:root:status: <1 0 3 7 1>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <1 0 3 4 1>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=2)
INFO:root:status: <1 0 1 4 1>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=1)
INFO:root:status: <1 0 1 4 0>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 0 1 4 0>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=2)
INFO:root:status: <0 0 1 2 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 0

## Automatic comparison of strategies

In [114]:
def compare_strategies(plyr1_strat, plyr2_strat, nim_size) :
    strategy = (plyr1_strat, plyr2_strat)
    plyr1_victories = 0
    plyr2_victories = 0
    for _ in tqdm(range(100)) :
        nim = Nim(nim_size)
        player = 1
        while nim:
            print(nim)
            ply = strategy[player](nim)
            print(ply)
            nim.nimming(ply)
            player = 1 - player
        if player == 0 :
            plyr1_victories += 1
        else :
            plyr2_victories += 1
    
    print("Victories -> Player 1 :", plyr1_victories, ", Player 2 :", plyr2_victories)

compare_strategies(optimal, optimal_rule_based1, 10)

  0%|          | 0/100 [00:00<?, ?it/s]

<1 3 5 7 9 11 13 15 17 19>
<1 3 5 7 9 11 13 15 17 19> added in cache --> {'possible_moves': {Nimply(row=0, num_objects=1): 3, Nimply(row=1, num_objects=1): 3, Nimply(row=1, num_objects=2): 0, Nimply(row=1, num_objects=3): 1, Nimply(row=2, num_objects=1): 3, Nimply(row=2, num_objects=2): 4, Nimply(row=2, num_objects=3): 5, Nimply(row=2, num_objects=4): 6, Nimply(row=2, num_objects=5): 7, Nimply(row=3, num_objects=1): 3, Nimply(row=3, num_objects=2): 0, Nimply(row=3, num_objects=3): 1, Nimply(row=3, num_objects=4): 6, Nimply(row=3, num_objects=5): 7, Nimply(row=3, num_objects=6): 4, Nimply(row=3, num_objects=7): 5, Nimply(row=4, num_objects=1): 3, Nimply(row=4, num_objects=2): 12, Nimply(row=4, num_objects=3): 13, Nimply(row=4, num_objects=4): 14, Nimply(row=4, num_objects=5): 15, Nimply(row=4, num_objects=6): 8, Nimply(row=4, num_objects=7): 9, Nimply(row=4, num_objects=8): 10, Nimply(row=4, num_objects=9): 11, Nimply(row=5, num_objects=1): 3, Nimply(row=5, num_objects=2): 0, Nimply(row

  0%|          | 0/100 [00:01<?, ?it/s]

<1 3 5 7 0 11 13 15 9 19> added in cache --> {'possible_moves': {Nimply(row=0, num_objects=1): 18, Nimply(row=1, num_objects=1): 18, Nimply(row=1, num_objects=2): 17, Nimply(row=1, num_objects=3): 16, Nimply(row=2, num_objects=1): 18, Nimply(row=2, num_objects=2): 21, Nimply(row=2, num_objects=3): 20, Nimply(row=2, num_objects=4): 23, Nimply(row=2, num_objects=5): 22, Nimply(row=3, num_objects=1): 18, Nimply(row=3, num_objects=2): 17, Nimply(row=3, num_objects=3): 16, Nimply(row=3, num_objects=4): 23, Nimply(row=3, num_objects=5): 22, Nimply(row=3, num_objects=6): 21, Nimply(row=3, num_objects=7): 20, Nimply(row=5, num_objects=1): 18, Nimply(row=5, num_objects=2): 17, Nimply(row=5, num_objects=3): 16, Nimply(row=5, num_objects=4): 31, Nimply(row=5, num_objects=5): 30, Nimply(row=5, num_objects=6): 29, Nimply(row=5, num_objects=7): 28, Nimply(row=5, num_objects=8): 27, Nimply(row=5, num_objects=9): 26, Nimply(row=5, num_objects=10): 25, Nimply(row=5, num_objects=11): 24, Nimply(row=6, n

AssertionError: 

## First task : Improve rule-based agent

In [13]:
def optimal_rule_based1(state : Nim) -> Nimply :
    # We keep the basic idea : we first have to filter the good moves before selecting one
    analysis = analize(state)
    good_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
   
    # If there is no good move, we can only select one random bad move
    if not good_moves:
        good_moves = list(analysis["possible_moves"].keys())
        ply = random.choice(good_moves)
        return ply
    
    # The best move one can make is the one that lets no good move to their opponent
    best_moves = list()
    for ply in good_moves :
        tmp_nim = deepcopy(state)
        tmp_nim.nimming(ply)
        opponent_moves = analize(tmp_nim)["possible_moves"]
        good_opponent_moves = [ply for ply,ns in opponent_moves.items() if ns != 0]
        if not good_opponent_moves :
            best_moves.append(ply)
    
    # If there are best moves, the agent can randomly choose one of them
    if len(best_moves) > 0 :
        ply = random.choice(best_moves)
        return ply
    
    # If there are no best moves, agent can only choose a good move
    ply = random.choice(good_moves)
    return ply

In [12]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, optimal_rule_based1)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <1 2 5 7 9>


AttributeError: 'NoneType' object has no attribute 'rows'